In [2]:
import pygeohash as gh
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%load_ext autoreload
%autoreload 2

sys.path.append(os.path.dirname(sys.path[0]))

from python.data_cleaning import prepare_data_for_groupby
from python.grouped_data import data_for_analysis
from python import mlmodel
import tensorflow as tf

2022-12-08 16:29:03.321662: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-08 16:29:03.507104: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-08 16:29:03.515198: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-08 16:29:03.515213: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

In [6]:
csv_path = '/home/linnanchen/code/MagicaBleps/UK-road-safety/raw_data'
df = pd.read_csv(os.path.join(csv_path,'dft-road-casualty-statistics-accident-1979-2021.csv'))
df.head()

/tmp/ipykernel_22888/212364011.py:2: DtypeWarning: Columns (0,2,15,16,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(csv_path,'dft-road-casualty-statistics-accident-1979-2021.csv'))


,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,...,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location
0,197901A11AD14,1979,01A11AD14,NaN,NaN,NaN,NaN,1,3,2,...,-1,1,8,1,-1,0,-1,-1,-1,-1
1,197901A1BAW34,1979,01A1BAW34,198460.0,894000.0,NaN,NaN,1,3,1,...,-1,4,8,3,-1,0,-1,-1,-1,-1
2,197901A1BFD77,1979,01A1BFD77,406380.0,307000.0,NaN,NaN,1,3,2,...,-1,4,8,3,-1,0,-1,-1,-1,-1
3,197901A1BGC20,1979,01A1BGC20,281680.0,440000.0,NaN,NaN,1,3,2,...,-1,4,8,3,-1,0,-1,-1,-1,-1
4,197901A1BGF95,1979,01A1BGF95,153960.0,795000.0,NaN,NaN,1,2,2,...,-1,4,3,3,-1,0,-1,-1,-1,-1


In [8]:
test=prepare_data_for_groupby(df,6)

In [9]:
test_data=data_for_analysis(test,granularity='W')
test_data.head(52)

,Accidents
date,
1999-01-04,1586
1999-01-11,4623
1999-01-18,5000
1999-01-25,4589
1999-02-01,4137
1999-02-08,3986
1999-02-15,4320
1999-02-22,3902
1999-03-01,4321


In [10]:
hashes=pd.DataFrame(test[['geohash','accident_year']].groupby('geohash').count())
hashes.columns=['Accidents']
hashes.sort_values('Accidents',inplace=True,ascending=False)
hashes.shape

(184578, 1)

In [11]:
hashes[hashes['Accidents']>=100]

,Accidents
geohash,
gcpvj0,2045
gcpvhc,1959
gcpvj4,1871
gcpvj1,1816
gcpuv2,1744
...,...
gcwzyg,100
gcx6bn,100
u120gg,100


In [12]:
#Isolating the square where there are more accidents
max_square=hashes[hashes['Accidents']==hashes.Accidents.max()].index[0]
max_square

'gcpvj0'

In [13]:
#Filtering the data for the max_square
test_max_square=test[test['geohash']==max_square]
test_max_square.shape

(2045, 13)

In [14]:
test_data_max_square=data_for_analysis(test_max_square,'W')
test_data_max_square.head(50)

,Accidents
date,
1999-01-04,1
1999-01-11,1
1999-01-18,1
1999-01-25,1
1999-02-01,1
1999-02-08,4
1999-02-15,3
1999-02-22,5
1999-03-01,2


In [15]:
FOLD_LENGTH=208 #each fold spans over 4 years
FOLD_STRIDE=52 #we have a  fold every 1 year
INPUT_LENGTH=10 #every X_i sequence is 10 weeks long
TRAIN_TEST_RATIO=.75 #in each fold, we use 3 years for training and 1 year for test
OUTPUT_LENGTH=10 #we want to predict the number of accidents in the next 10 weeks (y_i length)
SEQUENCE_STRIDE = 1
TARGET='Accidents'

In [16]:
folds=mlmodel.get_folds(test_data_max_square,FOLD_LENGTH,FOLD_STRIDE)
np.array(folds).shape

(20, 208, 1)

In [28]:
fold1=folds[0]
fold1

,Accidents
date,
1999-01-04,1
1999-01-11,1
1999-01-18,1
1999-01-25,1
1999-02-01,1
...,...
2002-11-25,2
2002-12-02,0
2002-12-09,1


In [29]:
(fold1_train, fold1_test) = mlmodel.train_test_split(fold1, TRAIN_TEST_RATIO, INPUT_LENGTH)
fold1_train.shape

(156, 1)

In [20]:
X_train1, y_train1 = mlmodel.get_X_y_strides(fold1_train,INPUT_LENGTH,OUTPUT_LENGTH,SEQUENCE_STRIDE)

In [21]:
X_train1.shape

(137, 10, 1)

In [37]:
X_test1, y_test1 = mlmodel.get_X_y_strides(fold1_test,INPUT_LENGTH,OUTPUT_LENGTH,SEQUENCE_STRIDE)

In [38]:
X_test1.shape

(43, 10, 1)

In [52]:
model = mlmodel.init_model(X_train1)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 10, 60)            14880     
                                                                 
 lstm_3 (LSTM)               (None, 10, 20)            6480      
                                                                 
 dense_3 (Dense)             (None, 10, 10)            210       
                                                                 
 dropout_2 (Dropout)         (None, 10, 10)            0         
                                                                 
 dense_4 (Dense)             (None, 10, 5)             55        
                                                                 
 dropout_3 (Dropout)         (None, 10, 5)             0         
                                                                 
 dense_5 (Dense)             (None, 10, 1)            

In [53]:
model.save('../model/gcpvj0')

INFO:tensorflow:Assets written to: ../model/gcpvj0/assets


In [76]:
csv_path = '../notebooks/data'
np.save(os.path.join(csv_path,'X_test.npy'), X_test1)


In [4]:
model = tf.keras.models.load_model('../model/gcpvj0')
X_test = np.load('../raw_data/data/X_test.npy')
y_pred = model.predict(X_test)

2/2 [==============================] - 0s 6ms/step


In [8]:
y_pred.shape

(43, 10, 1)

In [9]:
y_pred[-1,:,:]

array([[-0.00261419],
       [-0.00522514],
       [-0.01057571],
       [-0.00958139],
       [-0.00895172],
       [-0.00957408],
       [-0.00813039],
       [-0.00445555],
       [-0.00145602],
       [ 0.00142782]], dtype=float32)